# Month by month opioid data analysis for Texas

In [1]:
import pandas as pd
import numpy as np
import gzip
import os
import glob
import datetime

## Analysis begins here

In [21]:
# Don't run this part!!! The raw data is on Shining's laptop locally

zipfiles = "/Users/yangshining/Desktop/2021 fall @duke/IDS 720/mid-sem project/data/opioid_shipment/texas/*.gz"

filelist = glob.glob(zipfiles)

select_data = []

for gzfile in filelist:
    raw = pd.read_csv(gzfile, iterator= True, chunksize = 10000, compression='gzip', error_bad_lines=False)


    for df in raw:

        df["TRANSACTION_DATE1"] = pd.to_datetime(df["TRANSACTION_DATE"], format='%m%d%Y') # Convert transaction data to standardized date time

        df["year"] = pd.DatetimeIndex(df["TRANSACTION_DATE1"]).year # Extract year

        df["month"] = pd.DatetimeIndex(df["TRANSACTION_DATE1"]).month # Extract month

        df["year_month"] = df['TRANSACTION_DATE1'].dt.strftime('%Y-%m') # Create year-month column

        df = df[["BUYER_COUNTY", "BUYER_STATE", "year", "month", "year_month", "MME"]] # Subset the columns that we need

        select_data.append(df) # Append the chunks to the list



whole_data = pd.concat(select_data) # Concatenate data

grouped1 = whole_data.groupby(["BUYER_STATE", "BUYER_COUNTY", "year", "month"], as_index=False).sum("MME")
# grouped1: groupby year and month

grouped2 = whole_data.groupby(["BUYER_STATE", "BUYER_COUNTY", "year_month"], as_index=False).sum("MME")
#grouped2: groupby year_month column


# Test if the data is correctly loaded

#print(len(grouped))

grouped1["BUYER_STATE"].value_counts()

/Users/yangshining/opt/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


TX    24042
OK     8292
LA     6834
NM     3156
Name: BUYER_STATE, dtype: int64

In [22]:
# test for duplication
assert not grouped1.duplicated(["BUYER_STATE", "BUYER_COUNTY", "year", "month"]).any()

In [28]:
grouped1.to_parquet("/Users/yangshining/Desktop/2021 fall @duke/IDS 720/mid-sem project/data/opioid_parquet/tx_yymm_separate.parquet", engine='fastparquet')

grouped2.to_parquet("/Users/yangshining/Desktop/2021 fall @duke/IDS 720/mid-sem project/data/opioid_parquet/tx_yymm_integrated.parquet", engine='fastparquet')

## Fill in population data of La Salle, LA

As we are now analyzing data on a monthly basis, missing a county's population could become a big problem (because the missing observations are now a lot). So I fill the missing values here.

In [2]:
# load population

population = pd.read_parquet("/Users/yangshining/Desktop/pds2021-opioids-pds6/10_modified_data/pop_final.parquet", engine='fastparquet')
population.sample(10)

,STATE,COUNTY,STNAME,CTYNAME,Year,Population,STATE1,COUNTY1,fips
2771,48,413,Texas,Schleicher County,2003,2958,48,413,48413
16409,13,115,Georgia,Floyd County,2008,95922,13,115,13115
5375,40,35,Oklahoma,Craig County,2004,14739,40,035,40035
1296,26,87,Michigan,Lapeer County,2003,90573,26,087,26087
38707,13,35,Georgia,Butts County,2015,23520,13,035,13035
9087,48,281,Texas,Lampasas County,2005,18906,48,281,48281
8975,48,57,Texas,Calhoun County,2005,20715,48,057,48057
13276,13,233,Georgia,Polk County,2007,40789,13,233,13233
27721,40,53,Oklahoma,Grant County,2011,4563,40,053,40053
28450,51,117,Virginia,Mecklenburg County,2011,32560,51,117,51117


In [3]:
# create rows for La salle, LA


# create a list of Series

listofSeries = [
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2003, 14356, "22", "059", "22059"],
        index=population.columns,
    ),
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2004, 14366, "22", "059", "22059"],
        index=population.columns,
    ),
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2005, 14313, "22", "059", "22059"],
        index=population.columns,
    ),
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2006, 14519, "22", "059", "22059"],
        index=population.columns,
    ),
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2007, 14570, "22", "059", "22059"],
        index=population.columns,
    ),
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2008, 14667, "22", "059", "22059"],
        index=population.columns,
    ),
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2009, 14717, "22", "059", "22059"],
        index=population.columns,
    ),
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2010, 14908, "22", "059", "22059"],
        index=population.columns,
    ),
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2011, 14941, "22", "059", "22059"],
        index=population.columns,
    ),
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2012, 14862, "22", "059", "22059"],
        index=population.columns,
    ),
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2013, 14821, "22", "059", "22059"],
        index=population.columns,
    ),
    pd.Series(
        [22, 59, "Louisiana", "La Salle Parish", 2014, 14875, "22", "059", "22059"],
        index=population.columns,
    )
]


# append these rows to population dataframe
population = population.append(listofSeries, ignore_index=True)


# check if the rows are correctly appended
population[
    (population["STNAME"] == "Louisiana")
    & (population["CTYNAME"] == "La Salle Parish")
]


,STATE,COUNTY,STNAME,CTYNAME,Year,Population,STATE1,COUNTY1,fips
41483,22,59,Louisiana,La Salle Parish,2003,14356,22,059,22059
41484,22,59,Louisiana,La Salle Parish,2004,14366,22,059,22059
41485,22,59,Louisiana,La Salle Parish,2005,14313,22,059,22059
41486,22,59,Louisiana,La Salle Parish,2006,14519,22,059,22059
41487,22,59,Louisiana,La Salle Parish,2007,14570,22,059,22059
41488,22,59,Louisiana,La Salle Parish,2008,14667,22,059,22059
41489,22,59,Louisiana,La Salle Parish,2009,14717,22,059,22059
41490,22,59,Louisiana,La Salle Parish,2010,14908,22,059,22059
41491,22,59,Louisiana,La Salle Parish,2011,14941,22,059,22059
41492,22,59,Louisiana,La Salle Parish,2012,14862,22,059,22059


In [4]:
# Unify the data type of some columns 

population["STATE"] = population["STATE"].astype(int)
population["COUNTY"] = population["COUNTY"].astype(int)
population["Year"] = population["Year"].astype(int)


In [5]:
# store the data

population.to_parquet("/Users/yangshining/Desktop/pds2021-opioids-pds6/10_modified_data/pop_add_lasalle.parquet", engine='fastparquet')

## Merge with FIPS

In [5]:
# load opioid data of Texas and control

opi_tx = pd.read_parquet("/Users/yangshining/Desktop/2021 fall @duke/IDS 720/mid-sem project/data/opioid_parquet/tx_yymm_separate.parquet", engine='fastparquet')

opi_tx.sample(10)

,BUYER_STATE,BUYER_COUNTY,year,month,MME
6891,NM,BERNALILLO,2010,10,3.717306e+07
8113,NM,LINCOLN,2011,5,9.621965e+05
7802,NM,GRANT,2014,8,1.362313e+06
22532,TX,COMAL,2009,8,1.597241e+06
17408,OK,STEPHENS,2014,3,2.756969e+06
32722,TX,MADISON,2013,1,5.474603e+04
20211,TX,BRAZORIA,2014,7,6.176671e+06
24860,TX,ELLIS,2012,12,3.828857e+06
3649,LA,NATCHITOCHES,2010,6,6.912529e+05
30971,TX,KINNEY,2011,2,2.569208e+04


In [6]:
# test for duplication

assert not opi_tx.duplicated(["BUYER_STATE", "BUYER_COUNTY", "year", "month"]).any()

In [7]:
# Load fips data
fips = pd.read_csv("https://raw.githubusercontent.com/kjhealy/fips-codes/master/county_fips_master.csv", encoding="ISO-8859-1")

# Modify the county name in fips for better merging with the opioid data
fips['county_name']= fips['county_name'].str[:-7]
fips['county_name']= fips['county_name'].str.upper()

fips.head()

,fips,county_name,state_abbr,state_name,long_name,sumlev,region,division,state,county,crosswalk,region_name,division_name
0,1001,AUTAUGA,AL,Alabama,Autauga County AL,50.0,3.0,6.0,1.0,1.0,3-6-1-1,South,East South Central
1,1003,BALDWIN,AL,Alabama,Baldwin County AL,50.0,3.0,6.0,1.0,3.0,3-6-1-3,South,East South Central
2,1005,BARBOUR,AL,Alabama,Barbour County AL,50.0,3.0,6.0,1.0,5.0,3-6-1-5,South,East South Central
3,1007,BIBB,AL,Alabama,Bibb County AL,50.0,3.0,6.0,1.0,7.0,3-6-1-7,South,East South Central
4,1009,BLOUNT,AL,Alabama,Blount County AL,50.0,3.0,6.0,1.0,9.0,3-6-1-9,South,East South Central


In [8]:
# This cell is to fix the county name inconsistency between the two datasets

# Fix ST JOHN THE BAPTIST
opi_tx['BUYER_COUNTY'] = np.where(opi_tx["BUYER_COUNTY"].str[:3] == "ST ", "ST. " + opi_tx["BUYER_COUNTY"].str[3:] , opi_tx["BUYER_COUNTY"])

# Fix SAINT

opi_tx['BUYER_COUNTY'] = np.where(opi_tx["BUYER_COUNTY"].str[:6] == "SAINT ", "ST. " + opi_tx["BUYER_COUNTY"].str[6:] , opi_tx["BUYER_COUNTY"])

#Fix DONA ANA

fips['county_name'] = np.where(fips['county_name'] == "DOÐA ANA", "DONA ANA" , fips['county_name'])

# Fix DE SOTO
opi_tx['BUYER_COUNTY'] = np.where(opi_tx["BUYER_COUNTY"] == "DE SOTO", "DESOTO" , opi_tx["BUYER_COUNTY"])
fips['county_name'] = np.where(fips['county_name'] == "DE SOTO", "DESOTO" , fips['county_name'])

# Fix DE KALB
opi_tx['BUYER_COUNTY'] = np.where(opi_tx["BUYER_COUNTY"] == "DE KALB", "DEKALB" , opi_tx["BUYER_COUNTY"])

# Fix DE WITT
opi_tx['BUYER_COUNTY'] = np.where(opi_tx["BUYER_COUNTY"] == "DE WITT", "DEWITT" , opi_tx["BUYER_COUNTY"])


In [9]:
# merging

# Subset the fips data - keep only the columns we need 
fips_sub = fips[["county_name", "state_abbr", "fips"]]

opi_merge = pd.merge(fips_sub, opi_tx, how = "right",left_on = ["state_abbr", "county_name"], right_on=["BUYER_STATE", "BUYER_COUNTY"])

# Test if there's any na values
assert len(opi_merge[opi_merge["fips"].isna()]) ==0

In [10]:
# load population data after appending the rows of La Salle, Louisiana

population = pd.read_parquet("/Users/yangshining/Desktop/pds2021-opioids-pds6/10_modified_data/pop_add_lasalle.parquet", engine='fastparquet')

In [22]:
# Change the population FIPS code to strings
population["fips"] = population["fips"].astype(str)

# Modify data types of columns - prepare for merging

population["Year"] = population["Year"].astype(int)

opi_merge["fips"] = opi_merge["fips"].astype(str)

opi_merge["fips"] = opi_merge["fips"].apply(lambda x: x.zfill(5)) # fill fips code up to 5 digits

opi_tx_pop = pd.merge(opi_merge, population, left_on=["fips", "year"], right_on=["fips", "Year"], how='left') #Merging based on opioid dataset



# Check if there's any missing value in population

,county_name,state_abbr,fips,BUYER_STATE,BUYER_COUNTY,year,month,MME,STATE,COUNTY,STNAME,CTYNAME,Year,Population,STATE1,COUNTY1


In [23]:
# keep the columns that we need

opi_tx_pop = opi_tx_pop[["BUYER_STATE", "BUYER_COUNTY", "year", "month", "MME", "Population"]]

# Generate MME per cap

opi_tx_pop["MME_per_cap"] = opi_tx_pop["MME"]/opi_tx_pop["Population"]

In [25]:
# store the data

opi_tx_pop.to_parquet("/Users/yangshining/Desktop/pds2021-opioids-pds6/10_modified_data/opi_tx_pop.parquet", engine='fastparquet')